In [ ]:
# Setup
import pandas as pd
import sqlite3
import json
from pathlib import Path
from collections import defaultdict

print("Export and analysis setup complete")


In [ ]:
# Load Data from Database
db_path = Path("data/processed_messages.db")

if not db_path.exists():
    print("Database not found! Please run notebook 04 first.")
    print("Creating sample data for demo...")
    
    # Create sample data for demo
    db_path.parent.mkdir(parents=True, exist_ok=True)
    with sqlite3.connect(db_path) as conn:
        cursor = conn.cursor()
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS processed_messages (
                id INTEGER, channel TEXT, channel_title TEXT, original_text TEXT,
                cleaned_text TEXT, date TEXT, views INTEGER, entities TEXT,
                PRIMARY KEY (id, channel)
            )
        """)
        
        # Insert sample data
        sample_data = [
            (123, '@ShegerOnlineStore', 'Sheger Online Shopping', 
             'የሴቶች ቦርሳ ዋጋ 2500 ብር በአዲስ አበባ', 
             'የሴቶች ቦርሳ ዋጋ 2500 ብር በአዲስ አበባ',
             '2024-01-01T10:00:00', 150, 
             '{"prices": ["2500"], "locations": ["አዲስ አበባ"], "products": ["ቦርሳ"]}'),
            (124, '@ShegerOnlineStore', 'Sheger Online Shopping',
             'ሞባይል ፎን 15000 ብር delivery ከነ ቦሌ',
             'ሞባይል ፎን 15000 ብር delivery ከነ ቦሌ', 
             '2024-01-01T11:00:00', 200,
             '{"prices": ["15000"], "locations": ["ቦሌ"], "products": ["ሞባይል", "ፎን"]}')
        ]
        
        for data in sample_data:
            cursor.execute("""
                INSERT OR REPLACE INTO processed_messages 
                VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            """, data)
        conn.commit()

# Load data from database
with sqlite3.connect(db_path) as conn:
    df = pd.read_sql_query("SELECT * FROM processed_messages", conn)

print(f"Loaded {len(df)} processed messages from database")
print(f"Channels: {df['channel'].unique().tolist()}")

# Parse entities JSON
df['entities_parsed'] = df['entities'].apply(json.loads)

print("\nSample data:")
print(df[['channel', 'original_text', 'entities']].head())


In [ ]:
# Analyze Entities Across Channels
print("Entity Analysis Across Channels:")
print("=" * 40)

# Collect all entities by type
all_entities = defaultdict(list)
channel_stats = {}

for _, row in df.iterrows():
    channel = row['channel']
    entities = row['entities_parsed']
    
    if channel not in channel_stats:
        channel_stats[channel] = {
            'messages': 0,
            'prices': set(),
            'products': set(), 
            'locations': set()
        }
    
    channel_stats[channel]['messages'] += 1
    
    for entity_type, entity_list in entities.items():
        all_entities[entity_type].extend(entity_list)
        channel_stats[channel][entity_type].update(entity_list)

# Print channel statistics
for channel, stats in channel_stats.items():
    print(f"\n{channel}:")
    print(f"  Messages: {stats['messages']}")
    print(f"  Unique prices: {len(stats['prices'])} - {sorted(stats['prices'])}")
    print(f"  Unique products: {len(stats['products'])} - {list(stats['products'])}")
    print(f"  Unique locations: {len(stats['locations'])} - {list(stats['locations'])}")

# Overall statistics
print(f"\nOverall Statistics:")
print(f"  Total channels: {len(channel_stats)}")
print(f"  Total messages: {len(df)}")
print(f"  Unique prices: {len(set(all_entities['prices']))} - {sorted(set(all_entities['prices']))}")
print(f"  Unique products: {len(set(all_entities['products']))} - {list(set(all_entities['products']))}")
print(f"  Unique locations: {len(set(all_entities['locations']))} - {list(set(all_entities['locations']))}")


In [ ]:
# Export to CSV
export_dir = Path("data/processed")
export_dir.mkdir(parents=True, exist_ok=True)

# Prepare data for CSV export
csv_data = []
for _, row in df.iterrows():
    entities = row['entities_parsed']
    csv_row = {
        'message_id': row['id'],
        'channel': row['channel'],
        'channel_title': row['channel_title'],
        'text': row['original_text'],
        'cleaned_text': row['cleaned_text'],
        'date': row['date'],
        'views': row['views'],
        'products': ', '.join(entities['products']),
        'prices': ', '.join(entities['prices']),
        'locations': ', '.join(entities['locations']),
        'entities_json': row['entities']
    }
    csv_data.append(csv_row)

# Create DataFrame and export
export_df = pd.DataFrame(csv_data)
csv_path = export_dir / "processed_messages.csv"
export_df.to_csv(csv_path, index=False)

print(f"Exported data to: {csv_path}")
print(f"CSV contains {len(export_df)} rows from {len(df['channel'].unique())} channels")

# Show preview
print(f"\nCSV Preview:")
preview_cols = ['channel', 'text', 'products', 'prices', 'locations']
print(export_df[preview_cols].to_string(max_colwidth=40, index=False))


In [ ]:
# Create Channel Summary Report
summary_data = []
for channel, stats in channel_stats.items():
    summary_row = {
        'channel': channel,
        'channel_title': df[df['channel'] == channel]['channel_title'].iloc[0],
        'message_count': stats['messages'],
        'unique_prices': len(stats['prices']),
        'unique_products': len(stats['products']),
        'unique_locations': len(stats['locations']),
        'prices': ', '.join(sorted(stats['prices'])),
        'products': ', '.join(stats['products']),
        'locations': ', '.join(stats['locations'])
    }
    summary_data.append(summary_row)

summary_df = pd.DataFrame(summary_data)
summary_path = export_dir / "channel_summary.csv"
summary_df.to_csv(summary_path, index=False)

print(f"Exported channel summary to: {summary_path}")
print(f"\nChannel Summary:")
print(summary_df[['channel', 'message_count', 'unique_prices', 'unique_products', 'unique_locations']].to_string(index=False))

print(f"\nTask 1 Complete!")
print(f"Data files created:")
print(f"  - Database: {db_path}")
print(f"  - Messages CSV: {csv_path}")
print(f"  - Summary CSV: {summary_path}")
print(f"\nReady for Task 2: CoNLL labeling and NER training!")
